In [22]:
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re

In [ ]:
# Initialize the WebDriver
driver = webdriver.Chrome()

# URL of the login page
login_url = "your url"
driver.get(login_url)

# Wait for the login page to load
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, 'user-email'))
)

# Input username and password
username = "your email"
password = "your password"

username_field = driver.find_element(By.ID, 'user-email')
password_field = driver.find_element(By.ID, 'user-password')

username_field.send_keys(username)
password_field.send_keys(password)

# Submit the login form
login_button = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[2]/div/div/div/div/div/div/form/div/div/div[3]/div/div/button') #id of my login button
login_button.click()

# Wait for the main page to load after login
main_page_url = "your url"
WebDriverWait(driver, 10).until(
    EC.url_to_be(main_page_url)
)

In [24]:
hrefs = []

def extract_hrefs():
    try:
        tbody = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[1]/main/div[2]/div/div/div/div/div[2]/table/tbody')) #fetching table body
            
        # //*[@id="main-container"]/div/div[2]/div/div/div[2]/table/tbody
        )
        rows = tbody.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            status_element = row.find_element(By.XPATH, './/span[@data-testid="display-status"]')
            status = status_element.text.strip()
            if status == "Sent": # only extracting emails which have been sent
                a_elements = row.find_elements(By.TAG_NAME, 'a')
                for a in a_elements:
                    href = a.get_attribute('href') #extracting and storing email specific links
                    if href and href not in hrefs:
                        hrefs.append(href)
    except Exception as e:
        print(f"Error extracting hrefs: {e}")

def get_page_content():
    try:
        tbody = driver.find_element(By.XPATH, '//*[@id="app"]/div[1]/main/div[2]/div/div/div/div/div[2]/table/tbody')
        return tbody.text
    except Exception as e:
        print(f"Error getting page content: {e}")
        return ""

# Extract hrefs from the first page
extract_hrefs()

In [ ]:
# Loop to click 'Next' button and extract hrefs from subsequent pages
page_num = 1
previous_page_content = get_page_content()

while True:
    try:
        # Wait for the 'Next' button to be clickable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "btn") and .//i[contains(@class, "fa-chevron-right")]]'))
        ) # extracting next button through x path

        # Click the 'Next' button
        next_button.click()
        page_num += 1
        print(f"Extracting hrefs from page {page_num}")
        
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[1]/main/div[2]/div/div/div/div/div[2]/table/tbody'))
        )

        current_page_content = get_page_content()
        
        if current_page_content == previous_page_content:
            print("Reached the last page. Exiting loop.")
            break

        previous_page_content = current_page_content

        extract_hrefs()
    except Exception as e:
        print(f"No more pages or encountered an error: {e}")
        break

# Print all extracted hrefs
for href in hrefs:
    print(href)


In [ ]:
# Function to scrape data from each page
def scrape_data(url):
    driver.get(url)
    try:
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'row.items-push'))
        )

        data = {}
        parent_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//label[text()="Email Marketing"]/..'))
        )
        title_element = parent_element.find_element(By.XPATH, './/span[@data-testid="display-name"]')
        data['Title'] = title_element.text.strip()
        data['Subject'] = "Subject not found or empty"  # Default value
        recipients_text = driver.find_element(By.CLASS_NAME, 'col-12.font-size-h5.font-w500').text.strip()
        recipients_num = re.search(r'\((\d{1,3}(?:,\d{3})*) Recipients\)', recipients_text)
        if recipients_num:
            data['Total Recipients'] = recipients_num.group(1).replace(',', '')  
        else:
            data['Total Recipients'] = "0"

        # Extract Successful Deliveries
        deliveries_text = driver.find_element(By.XPATH, '//a[contains(text(),"Delivered")]/following-sibling::div').text.strip()
        deliveries_match = re.match(r'\d+% \(([\d,]+)\)', deliveries_text)
        if deliveries_match:
            data['Successful Deliveries'] = deliveries_match.group(1).replace(',', '')
            
        opened_text = driver.find_element(By.XPATH, '//a[contains(text(),"Opened")]/following-sibling::div').text.strip()
        opened_match = re.match(r'(\d+)% \(([\d,]+)\)', opened_text)
        if opened_match:
            data['Open Rate'] = opened_match.group(1) + '%'
            data['Total Opens'] = opened_match.group(2).replace(',', '')
            

        clicked_text = driver.find_element(By.XPATH, '//a[contains(text(),"Clicked")]/following-sibling::div').text.strip()
        clicked_match = re.match(r'(\d+)% \(([\d,]+)\)', clicked_text)
        if clicked_match:
            data['Click Rate'] = clicked_match.group(1) + '%'
            data['Total Clicks'] = clicked_match.group(2).replace(',', '')
            
        data['Unsubscribes'] = driver.find_element(By.XPATH, '//a[contains(text(),"Unsubscribed")]/preceding-sibling::div').text.strip()
        data['Total Bounces'] = driver.find_element(By.XPATH, '//a[contains(text(),"Bounced")]/preceding-sibling::div').text.strip()
        data['Unopened'] = driver.find_element(By.XPATH, '//a[contains(text(),"Unopened")]/preceding-sibling::div').text.strip()
        sent_datetime = driver.find_element(By.XPATH, '//div[@class="mt-2"]/span[@class="text-success"]').text.strip()
        date, time = sent_datetime.split(' - ')
        datetime_obj = datetime.strptime(f"{date} {time}", '%b %d, %Y %I:%M %p')
        data['Send Date'] = datetime_obj.strftime('%Y-%m-%d')
        data['Time'] = datetime_obj.strftime('%H:%M:%S')
        data['Send Weekday'] = datetime_obj.strftime('%A')
        try:
            raised_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//div[text()="Raised"]/preceding-sibling::div'))
            )
            data['Raised'] = raised_element.text.strip()
        except NoSuchElementException:
            print(f"'Raised' data not found for {url}")
            data['Raised'] = 'N/A' 

        # Click the 'Content' tab
        try:
            content_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "/content") and contains(@class, "tw-relative")]'))
            )
            content_tab.click()
        except TimeoutException:
            print("Content tab not found or not clickable.")
    
        # fetch the subject
        try:
            subject_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//span[@data-testid="display-subject"]'))
            )
            data['Subject'] = subject_element.text.strip()
        except TimeoutException:
            data['Subject'] = "Subject not found or empty"

        # Use CSS selectors 
        # data['Delivered'] = driver.find_element(By.CSS_SELECTOR, 'a.font-w500:contains("Delivered") + div').text.strip()
        # data['Opened'] = driver.find_element(By.CSS_SELECTOR, 'a.font-w500:contains("Opened") + div').text.strip()
        # data['Clicked'] = driver.find_element(By.CSS_SELECTOR, 'a.font-w500:contains("Clicked") + div').text.strip()
        # data['Unsubscribed'] = driver.find_element(By.CSS_SELECTOR, 'a.font-w500:contains("Unsubscribed") + div').text.strip()
        # data['Bounced'] = driver.find_element(By.CSS_SELECTOR, 'a.font-w500:contains("Bounced") + div').text.strip()
        # data['Raised'] = driver.find_element(By.XPATH, '//div[text()="Raised"]/preceding-sibling::div').text.strip()

        
        return data
    except Exception as e:
        print(f"Error scraping data from {url}: {e}")
        return None

# List to hold all scraped data
all_data = []

# Scrape data from each href
for href in hrefs:
    data = scrape_data(href)
    if data:
        all_data.append(data)

print(all_data)

In [27]:
# Convert the data to a DataFrame
df = pd.DataFrame(all_data)

# Save the data to an Excel file
df.to_excel('scraped_data.xlsx', index=False)

# Save the data to a CSV file
df.to_csv('scraped_data.csv', index=False)

In [ ]:
# Load the data
df1 = pd.read_excel(r"path of scraped data file")
df2 = pd.read_csv(r"path of my mailchimp file")

print("File 1")
print(df1.head())
print("\nFile 2")
print(df2.head())

# Identify common columns (excluding the key column 'Title')
common_columns = [col for col in df1.columns if col in df2.columns and col != 'Title']

# Include the 'Title' column in the list of columns to retain
columns_to_retain = ['Title'] + common_columns

# Subset both DataFrames to only include the columns to retain
df1_subset = df1[columns_to_retain]
df2_subset = df2[columns_to_retain]

# Merge the subset DataFrames on the common column 'Title'
merged_df = pd.merge(df1_subset, df2_subset, on='Title', how='outer', suffixes=('_df1', '_df2'))

# Combine the values for each common column
for col in common_columns:
    merged_df[col] = merged_df[f"{col}_df1"].combine_first(merged_df[f"{col}_df2"])
    merged_df.drop(columns=[f"{col}_df1", f"{col}_df2"], inplace=True)

# Reorder columns if necessary
columns_order = ['Title'] + common_columns
merged_df = merged_df[columns_order]

print("\nMerged Dataframe")
print(merged_df.head())

# Save the merged DataFrame to a new Excel file
merged_file_path = 'merged file path'
merged_df.to_excel(merged_file_path, index=False)

In [ ]:
# Close the WebDriver
driver.quit()